In [3]:
import gc   #garbage collector-to free memory
gc.enable() #Activate
import zipfile #to unzip files
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
!pip install --upgrade kaggle
from google.colab import files
from datetime import datetime
api_token=files.upload()

     |████████████████████████████████| 61kB 3.2MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=c5fbc061cefa33006d083600ce120655a007df8502ee730fcd5a7affa3f2d8b2
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


Saving kaggle.json to kaggle.json


In [ ]:
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c instacart-market-basket-analysis

 94% 185M/196M [00:01<00:00, 124MB/s]
100% 196M/196M [00:01<00:00, 120MB/s]


In [ ]:
with zipfile.ZipFile('/content/instacart-market-basket-analysis.zip','r')as z:
  z.extractall('.')

In [ ]:
with zipfile.ZipFile('/content/orders.csv.zip','r')as z:
    z.extractall('.')

In [ ]:
orders=pd.read_csv('./orders.csv')
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
import numpy as np
orders['days_since_prior_order'].replace(to_replace=np.nan,value=0,inplace=True)
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,0.0
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
#selecting only train data
tr_ord=orders[orders.eval_set=='train']
tr_ord.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,train,11,4,8,14.0
25,1492625,2,train,15,1,11,30.0
49,2196797,5,train,5,0,11,6.0
74,525192,7,train,21,2,11,6.0
78,880375,8,train,4,1,14,10.0


In [ ]:
with zipfile.ZipFile('/content/order_products__train.csv.zip','r')as z:
    z.extractall('.')

In [ ]:
train_orders=pd.read_csv('/content/order_products__train.csv')
train_orders.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Merging data frame with users'feature**

In [ ]:
user=pd.read_csv('/content/drive/MyDrive/Instacart/user.csv')
user.drop('Unnamed: 0',axis=1,inplace=True)
user.head()

,user_id,Total orders,user_re_ratio,user_ntre_ratio
0,1,10,0.694915,0.305085
1,2,14,0.476923,0.523077
2,3,12,0.625000,0.375000
3,4,5,0.055556,0.944444
4,5,4,0.378378,0.621622


In [ ]:
tr_ord=tr_ord.merge(user,on='user_id',how='left')
tr_ord.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,Total orders,user_re_ratio,user_ntre_ratio
0,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085
1,1492625,2,train,15,1,11,30.0,14,0.476923,0.523077
2,2196797,5,train,5,0,11,6.0,4,0.378378,0.621622
3,525192,7,train,21,2,11,6.0,20,0.669903,0.330097
4,880375,8,train,4,1,14,10.0,3,0.265306,0.734694


**Merging data frame with user-products'feature**

Here products are previously ordered products of a user

In [ ]:
user_prd=pd.read_csv('/content/drive/MyDrive/Instacart/uxp.csv')
user_prd.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
user_prd.head()

,user_id,product_id,user_prd_re_ratio,times_cart_123,times_prd_last_5
0,1,196,1.000000,9.0,5.0
1,1,10258,1.000000,7.0,5.0
2,1,10326,0.166667,0.0,0.0
3,1,12427,1.000000,8.0,5.0
4,1,13032,0.333333,0.0,2.0


In [ ]:
tr_ord=tr_ord.merge(user_prd,on=['user_id'],how='left')
tr_ord.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,Total orders,user_re_ratio,user_ntre_ratio,product_id,user_prd_re_ratio,times_cart_123,times_prd_last_5
0,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,196,1.000000,9.0,5.0
1,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,10258,1.000000,7.0,5.0
2,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,10326,0.166667,0.0,0.0
3,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,12427,1.000000,8.0,5.0
4,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,13032,0.333333,0.0,2.0


**Merging data frame with products' feature**

In [ ]:
prd=pd.read_csv('/content/drive/MyDrive/Instacart/prd.csv')
prd.drop('Unnamed: 0',axis=1,inplace=True)
prd.head()

,product_id,prd_cart_last_5,prd_ntre_ratio
0,1,5.4,0.386609
1,2,11.6,0.866667
2,3,4.4,0.267148
3,4,16.4,0.553191
4,5,8.2,0.400000


In [ ]:
tr_ord=tr_ord.merge(prd,on='product_id',how='left')
tr_ord.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,Total orders,user_re_ratio,user_ntre_ratio,product_id,user_prd_re_ratio,times_cart_123,times_prd_last_5,prd_cart_last_5,prd_ntre_ratio
0,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,196,1.000000,9.0,5.0,6.4,0.223520
1,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,10258,1.000000,7.0,5.0,4.4,0.286228
2,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,10326,0.166667,0.0,0.0,2.6,0.347991
3,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,12427,1.000000,8.0,5.0,2.6,0.259265
4,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,13032,0.333333,0.0,2.0,1.4,0.342842


**Merging data frame with user-dow feature**

In [ ]:
dow=pd.read_csv('/content/drive/MyDrive/Instacart/uxd.csv')
dow.drop('Unnamed: 0',axis=1,inplace=True)
dow.head()

,user_id,order_dow,dow_re_ratio,dow_ntre_ratio
0,1,1,0.882353,0.396157
1,1,2,0.444444,0.410229
2,1,3,0.545455,0.413728
3,1,4,0.727273,0.409021
4,2,1,0.671875,0.396157


In [ ]:
tr_ord=tr_ord.merge(dow,on=['user_id','order_dow'],how='left')
tr_ord.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,Total orders,user_re_ratio,user_ntre_ratio,product_id,user_prd_re_ratio,times_cart_123,times_prd_last_5,prd_cart_last_5,prd_ntre_ratio,dow_re_ratio,dow_ntre_ratio
0,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,196,1.000000,9.0,5.0,6.4,0.223520,0.727273,0.409021
1,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,10258,1.000000,7.0,5.0,4.4,0.286228,0.727273,0.409021
2,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,10326,0.166667,0.0,0.0,2.6,0.347991,0.727273,0.409021
3,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,12427,1.000000,8.0,5.0,2.6,0.259265,0.727273,0.409021
4,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,13032,0.333333,0.0,2.0,1.4,0.342842,0.727273,0.409021


To get class label or reordered info - we merge above df with train_orders on order_id and product_id

In [ ]:
tr_ord=tr_ord.merge(train_orders,on=['product_id','order_id'],how='left')

In [ ]:
tr_ord.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,Total orders,user_re_ratio,user_ntre_ratio,product_id,user_prd_re_ratio,times_cart_123,times_prd_last_5,prd_cart_last_5,prd_ntre_ratio,dow_re_ratio,dow_ntre_ratio,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,196,1.000000,9.0,5.0,6.4,0.223520,0.727273,0.409021,1.0,1.0
1,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,10258,1.000000,7.0,5.0,4.4,0.286228,0.727273,0.409021,6.0,1.0
2,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,10326,0.166667,0.0,0.0,2.6,0.347991,0.727273,0.409021,NaN,NaN
3,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,12427,1.000000,8.0,5.0,2.6,0.259265,0.727273,0.409021,NaN,NaN
4,1187899,1,train,11,4,8,14.0,10,0.694915,0.305085,13032,0.333333,0.0,2.0,1.4,0.342842,0.727273,0.409021,7.0,1.0


In [5]:
tr_ord.drop(['eval_set','order_number','order_dow','add_to_cart_order'],axis=1,inplace=True)
tr_ord=tr_ord.fillna(0)
tr_ord.isna().sum()

user_id                   0
order_hour_of_day         0
days_since_prior_order    0
Total orders              0
user_re_ratio             0
user_ntre_ratio           0
product_id                0
user_prd_re_ratio         0
times_cart_123            0
times_prd_last_5          0
prd_cart_last_5           0
prd_ntre_ratio            0
dow_re_ratio              0
dow_ntre_ratio            0
reordered                 0
order_id                  0
dtype: int64

In [ ]:
tr_ord.head()

,user_id,order_hour_of_day,days_since_prior_order,Total orders,user_re_ratio,user_ntre_ratio,product_id,user_prd_re_ratio,times_cart_123,times_prd_last_5,prd_cart_last_5,prd_ntre_ratio,dow_re_ratio,dow_ntre_ratio,reordered,order_id
0,1,8,14.0,10,0.694915,0.305085,196,1.000000,9.0,5.0,6.4,0.223520,0.727273,0.409021,1.0,1187899
1,1,8,14.0,10,0.694915,0.305085,10258,1.000000,7.0,5.0,4.4,0.286228,0.727273,0.409021,1.0,1187899
2,1,8,14.0,10,0.694915,0.305085,10326,0.166667,0.0,0.0,2.6,0.347991,0.727273,0.409021,0.0,1187899
3,1,8,14.0,10,0.694915,0.305085,12427,1.000000,8.0,5.0,2.6,0.259265,0.727273,0.409021,0.0,1187899
4,1,8,14.0,10,0.694915,0.305085,13032,0.333333,0.0,2.0,1.4,0.342842,0.727273,0.409021,1.0,1187899


In [ ]:
tr_ord.to_csv('/content/drive/MyDrive/Instacart/train_final.csv')

As we have seen in EDA that each user_id has only 1 train order_id (proved below also)  

Here in above df we hv two composite keys user_id+product_id and order_id+product_id so, we can remove one .

In [ ]:
#consider orders df
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,0.0
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
ord_id=orders[orders.eval_set=='train'][['user_id','order_id']]  #ids of train orders only
ord_id.head()

,user_id,order_id
10,1,1187899
25,2,1492625
49,5,2196797
74,7,525192
78,8,880375


In [ ]:
len(ord_id['user_id'].unique())

131209

In [ ]:
len(ord_id['order_id'].unique())

131209

In [ ]:
ord_id.shape

(131209, 2)

this can be clearly noticed here also that each train user_id has only 1 train order .

In [ ]:
tr_ord.drop(['order_id'],axis=1,inplace=True)
tr_ord.head()

,user_id,order_hour_of_day,days_since_prior_order,Total orders,user_re_ratio,user_ntre_ratio,product_id,user_prd_re_ratio,times_cart_123,times_prd_last_5,prd_cart_last_5,prd_ntre_ratio,dow_re_ratio,dow_ntre_ratio,reordered
0,1,8,14.0,10,0.694915,0.305085,196,1.000000,9.0,5.0,6.4,0.223520,0.727273,0.409021,1.0
1,1,8,14.0,10,0.694915,0.305085,10258,1.000000,7.0,5.0,4.4,0.286228,0.727273,0.409021,1.0
2,1,8,14.0,10,0.694915,0.305085,10326,0.166667,0.0,0.0,2.6,0.347991,0.727273,0.409021,0.0
3,1,8,14.0,10,0.694915,0.305085,12427,1.000000,8.0,5.0,2.6,0.259265,0.727273,0.409021,0.0
4,1,8,14.0,10,0.694915,0.305085,13032,0.333333,0.0,2.0,1.4,0.342842,0.727273,0.409021,1.0


In [ ]:
tr_ord.shape

(8474661, 15)

In [ ]:
tr_ord.to_csv('/content/drive/MyDrive/Instacart/train.csv')

We use train.csv for modelling. But since we hv to submit using order_id not user_id train_final.csv is useful there.